In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

In [ ]:
def import_data():
  global data_in, data_out
  data_in=np.array(pd.read_csv("./Preserved/div_U_2.csv", header=None), dtype=np.float32)
  data_out=np.array(pd.read_csv("./Preserved/P.csv", header=None), dtype=np.float32)
  data_in=data_in[:,:961]
  data_in=data_in[1:]-data_in[:-1]
  data_out=data_out[:,:961]
  data_out=data_out[1:]-data_out[:-1]
  data_in=data_in[1000:1100]
  data_out=data_out[1000:1100]
  data_in=data_in
  data_out=data_out

In [ ]:
tmp=torch.rand(1, 1, 5, 5)
print(tmp)
print(tmp[0, 0, :, 0])
print(tmp[0, 0, :, -1])
print(tmp[0, 0, 0, :])
print(tmp[0, 0, -1, :])
val=torch.sum(torch.abs(tmp[0, 0, :, 0]+tmp[0, 0, :, -1]+tmp[0, 0, 0, :]+tmp[0, 0, -1, :]))
print(val)

In [ ]:
import_data()
x_in=torch.tensor(data_in)
y_in=torch.tensor(data_out)
print(x_in.size())
print(y_in.size())
x=x_in.to(device)
y=y_in.to(device)
x= torch.nn.Sequential(torch.nn.Unflatten(1, (1,31,31)))(x)
y= torch.nn.Sequential(torch.nn.Unflatten(1, (1,31,31)))(y)
print(x.size())
print(y.size())
data_loader=DataLoader(TensorDataset(x,y), batch_size=50)

In [ ]:
d=31
class VAE(nn.Module):
    def __init__(self, imgChannels=1, featureDim=64*d*d, zDim=256):
        super(VAE, self).__init__()

        self.encConv1 = nn.Conv2d(imgChannels, 8, 3, padding=1)
        self.encConv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.encConv3 = nn.Conv2d(16, 64, 3, padding=1)
        self.encFC1 = nn.Linear(featureDim, zDim)
        self.encFC2 = nn.Linear(featureDim, zDim)

        self.decFC1 = nn.Linear(zDim, featureDim)
        self.decConv1 = nn.ConvTranspose2d(64, 16, 3, padding=1)
        self.decConv2 = nn.ConvTranspose2d(16, 8, 3, padding=1)
        self.decConv3 = nn.ConvTranspose2d(8, imgChannels, 3, padding=1)

    def encoder(self, x):
        x = F.relu(self.encConv1(x))
        x = F.relu(self.encConv2(x))
        x = F.relu(self.encConv3(x))
        x = x.view(-1, 64*d*d)
        mu = self.encFC1(x)
        #logVar = self.encFC2(x)
#         return mu, logVar
        return mu

    def reparameterize(self, mu, logVar):
        std = torch.exp(logVar/2)
        eps = torch.randn_like(std)
        return mu + std * eps

    def decoder(self, z):
        x = F.relu(self.decFC1(z))
        x = x.view(-1, 64, d, d)
        x = F.relu(self.decConv1(x))
        x = F.relu(self.decConv2(x))
        x = F.tanh(self.decConv3(x))
        return x

    def forward(self, x):

#         mu, logVar = self.encoder(x)
        mu = self.encoder(x)
#         z = self.reparameterize(mu, logVar)
#         out = self.decoder(z)
        out = self.decoder(mu)
#         return out, mu, logVar
        return out

In [ ]:
batch_size = 128
learning_rate = 1e-4
num_epochs = 10

In [ ]:
net = VAE().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.8)

In [ ]:
for epoch in range(5000):
    scheduler.step()
    for idx, (data, label) in enumerate(data_loader):
        imgs = data[0]
        lbls = label[0]
#         out, mu, logVar = net(imgs)
        out = net(imgs)

        outimg = out.detach().cpu().numpy()
#         kl_divergence = 0.5 * torch.sum(-1 - logVar + mu.pow(2) + logVar.exp())
#         loss = F.binary_cross_entropy(out, imgs, size_average=False) + kl_divergence
#         loss = F.binary_cross_entropy(out, imgs, size_average=False)

        
        # Compute laplacian
        y= torch.flatten(out, 1)
        y= torch.nn.Sequential(torch.nn.Unflatten(1, (31,31)))(y)
        
        z=torch.concat([y[:, :, :1], y], axis=2)[:, :, :31]
        w=torch.concat([y, y[:, :, 30:]], axis=2)[:, :, 1:]
        m=torch.concat([y[:, :1, :], y], axis=1)[:, :31, :]
        n=torch.concat([y, y[:, 30:, :]], axis=1)[:, 1:, :]
        L_y_pred=(z+w+m+n-4*y)*15.5*0.001 # h=1/31 /2h=>*15.5
        L_y_pred=torch.flatten(L_y_pred, start_dim=1)
        
        L_y_pred= torch.flatten(L_y_pred, 1)
        L_y_pred= torch.nn.Sequential(torch.nn.Unflatten(1, (1,31,31)))(L_y_pred)
        lyimg = L_y_pred.detach().cpu().numpy()
        

#         loss = F.mse_loss(out, lbls)
        loss = F.mse_loss(out, imgs)
#         loss = F.mse_loss(L_y_pred, imgs) + F.mse_loss(out, lbls)
        
        # BC enforcement
        val=torch.sum(torch.abs(out[0, 0, :, 0]+out[0, 0, :, -1]+out[0, 0, 0, :]+out[0, 0, -1, :]))
        loss=loss + F.mse_loss(val, torch.zeros(1))*5


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch {}: Loss {}'.format(epoch, loss))   
    
    # Monitering
    net.eval()
    with torch.no_grad():
        for (data, label) in random.sample(list(data_loader), 1):
            
            # item==data A
            imgs = data[0]
            imgs = imgs.to(device)
            img = np.transpose(imgs[0].cpu().numpy())
            # item==label B
            lbls = label[0]
            lbls = lbls.to(device)
            lbl = np.transpose(label[0].cpu().numpy())
            
            # print model_input A
            plt.subplot(141)
            plt.imshow(np.squeeze(img))
            
            # print model_output B
            out = net(imgs)
            outimg = np.transpose(out[0].cpu().numpy())
            plt.subplot(142)
#             plt.imshow(np.squeeze(outimg), vmin=np.min(img), vmax=np.max(img))
#             plt.imshow(np.squeeze(outimg), vmin=np.min(lbl), vmax=np.max(lbl))
            plt.imshow(np.squeeze(outimg))
            
            
            
            # print laplacian model_output A
            plt.subplot(143)
            arr=np.squeeze(outimg)
            arr_1=arr[0:-2, 1:-1]+arr[2:, 1:-1]+arr[1:-1, 0:-2]+arr[1:-1, 2:]-4*arr[1:-1, 1:-1]
            arr_1=arr_1*15.5*0.001
            plt.imshow(arr_1, vmin=np.min(img), vmax=np.max(img))
            
            
            # print gnd truth B
            plt.subplot(144)
            plt.imshow(np.squeeze(lbl), vmin=np.min(lbl), vmax=np.max(lbl))
            
            plt.show()
            plt.clf()
            plt.cla()
            plt.close("all")
            time.sleep(0.5)
            print("[check_data]max: ", np.max(img)) #
            print("[check_data]min: ", np.min(img))
            print("[check_data]max: ", np.max(outimg))
            print("[check_data]min: ", np.min(outimg))
            print("[check_data]max: ", np.max(arr_1)) #
            print("[check_data]min: ", np.min(arr_1))
            print("[check_data]max: ", np.max(lbl))
            print("[check_data]min: ", np.min(lbl))
            break
    

In [ ]:
torch.save(net, 'ConvAE_UnsupervisedPINN_BC_05.pth')